# Data Cleaning

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from google.colab import files

In [2]:
# Read data file
df = pd.read_csv('lighting_data.csv')

# Display data
df

,light_id,location_name,fault_type,timestamp,severity_level,fault_status,maintenance_cost
0,L0313,Jurong West,Power-related,2020-01-07 09:05:35,Low,Closed,297.45
1,L0385,Pasir Ris,Environmental,2020-01-07 10:28:12,Medium,In Progress,172.61
2,L0158,Canberra,Communication,2020-01-07 13:09:56,Medium,Detected,381.23
3,L0554,Jurong West,Communication,2020-01-08 01:55:51,Medium,In Progress,284.30
4,L0755,Bishan,Communication,2020-01-11 10:49:07,Critical,In Progress,76.52
...,...,...,...,...,...,...,...
163,L0768,Chinatown,Cybersecurity,2020-12-24 21:14:30,Critical,Detected,156.92
164,L0716,Pasir Ris,Control system,2020-12-26 03:30:47,High,Closed,NaN
165,L0085,Toa Payoh,Communication,2020-12-27 02:25:18,Medium,Closed,377.93
166,L0525,Pasir Ris,Control system,2020-12-28 00:39:48,Low,In Progress,191.01


In [3]:
# Dataset(rows, columns)
df.shape

(168, 7)

In [4]:
# Display first five data
df.head()

,light_id,location_name,fault_type,timestamp,severity_level,fault_status,maintenance_cost
0,L0313,Jurong West,Power-related,2020-01-07 09:05:35,Low,Closed,297.45
1,L0385,Pasir Ris,Environmental,2020-01-07 10:28:12,Medium,In Progress,172.61
2,L0158,Canberra,Communication,2020-01-07 13:09:56,Medium,Detected,381.23
3,L0554,Jurong West,Communication,2020-01-08 01:55:51,Medium,In Progress,284.30
4,L0755,Bishan,Communication,2020-01-11 10:49:07,Critical,In Progress,76.52


In [5]:
# Display last five data
df.tail()

,light_id,location_name,fault_type,timestamp,severity_level,fault_status,maintenance_cost
163,L0768,Chinatown,Cybersecurity,2020-12-24 21:14:30,Critical,Detected,156.92
164,L0716,Pasir Ris,Control system,2020-12-26 03:30:47,High,Closed,NaN
165,L0085,Toa Payoh,Communication,2020-12-27 02:25:18,Medium,Closed,377.93
166,L0525,Pasir Ris,Control system,2020-12-28 00:39:48,Low,In Progress,191.01
167,L0904,Outram Park,Environmental,2020-12-29 05:58:29,Medium,Detected,301.04


In [6]:
# Check data types & basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   light_id          168 non-null    object 
 1   location_name     168 non-null    object 
 2   fault_type        168 non-null    object 
 3   timestamp         168 non-null    object 
 4   severity_level    168 non-null    object 
 5   fault_status      168 non-null    object 
 6   maintenance_cost  150 non-null    float64
dtypes: float64(1), object(6)
memory usage: 9.3+ KB


In [7]:
# Find missing values
missing_count = df.isnull().sum()
missing_count

,0
light_id,0
location_name,0
fault_type,0
timestamp,0
severity_level,0
fault_status,0
maintenance_cost,18


The maintenance_cost column has 52 missing values.

In [8]:
# Find percentage of missing values
missing_count = df.isna().sum()
total_rows = len(df)
missing_proportion = (missing_count / total_rows) * 100

# Sort descending
missing_proportion = missing_proportion.round(2).sort_values(ascending=False)
print(missing_proportion)

maintenance_cost    10.71
location_name        0.00
light_id             0.00
fault_type           0.00
timestamp            0.00
severity_level       0.00
fault_status         0.00
dtype: float64


Since maintenance cost is a numeric column and may contain outliers, I will apply the median to fill its missing values.

In [9]:
# Find median of 'maintenance_cost' column
median_value = df['maintenance_cost'].median()
median_rounded = round(median_value, 2)
print("Median value:", median_rounded)

Median value: 262.23


In [10]:
# Fill missing values with median
df['maintenance_cost'].fillna(median_value, inplace=True)

/tmp/ipython-input-2732969241.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['maintenance_cost'].fillna(median_value, inplace=True)


In [11]:
# Check duplicate rows
num_duplicates = df.duplicated().sum()
print(f"Duplicate rows: {num_duplicates}")

Duplicate rows: 0


In [12]:
# Verify filling to ensure no null values
print(df['maintenance_cost'].isnull().sum())

0


In [13]:
# Convert timestamps to pandas datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort by timestamp
df = df.sort_values('timestamp').reset_index(drop=True)

In [14]:
# Verify data type
print(df['timestamp'].dtype)

datetime64[ns]


In [15]:
# Verify sort
is_sorted = df['timestamp'].equals(df['timestamp'].sort_values())
print("Is sorted:", is_sorted)

Is sorted: True


In [16]:
# Standardise categorical values
df['fault_type'] = df['fault_type'].str.strip().str.title()
df['severity_level'] = df['severity_level'].str.strip().str.title()
df['fault_status'] = df['fault_status'].str.strip().str.title()
df['location_name'] = df['location_name'].str.strip().str.title()

In [17]:
# Extract time features
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['day_of_week'] = df['timestamp'].dt.day_name()
df['hour'] = df['timestamp'].dt.hour

In [18]:
filename='lighting_data_cleaned.csv'

# Save to csv and download
df.to_csv(filename, index=False) # Save csv
files.download(filename) # Automatically download the file in colab

# Read final dataset
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,light_id,location_name,fault_type,timestamp,severity_level,fault_status,maintenance_cost,year,month,day,day_of_week,hour
0,L0313,Jurong West,Power-Related,2020-01-07 09:05:35,Low,Closed,297.45,2020,1,7,Tuesday,9
1,L0385,Pasir Ris,Environmental,2020-01-07 10:28:12,Medium,In Progress,172.61,2020,1,7,Tuesday,10
2,L0158,Canberra,Communication,2020-01-07 13:09:56,Medium,Detected,381.23,2020,1,7,Tuesday,13
3,L0554,Jurong West,Communication,2020-01-08 01:55:51,Medium,In Progress,284.30,2020,1,8,Wednesday,1
4,L0755,Bishan,Communication,2020-01-11 10:49:07,Critical,In Progress,76.52,2020,1,11,Saturday,10
...,...,...,...,...,...,...,...,...,...,...,...,...
163,L0768,Chinatown,Cybersecurity,2020-12-24 21:14:30,Critical,Detected,156.92,2020,12,24,Thursday,21
164,L0716,Pasir Ris,Control System,2020-12-26 03:30:47,High,Closed,262.23,2020,12,26,Saturday,3
165,L0085,Toa Payoh,Communication,2020-12-27 02:25:18,Medium,Closed,377.93,2020,12,27,Sunday,2
166,L0525,Pasir Ris,Control System,2020-12-28 00:39:48,Low,In Progress,191.01,2020,12,28,Monday,0
